# FSC curve of empiar10311_stack_crop.mrc VS several sigma values in Gaussian Denoising

In [ ]:
import mrcfile # pip install mrcfile
import matplotlib.pyplot as plt # pip install matplotlib
from self_fourier_shell_correlation import fsc_utils as fsc # pip install --upgrade --force-reinstall "self_fourier_shell_correlation @ git+https://github.com/vicente-gonzalez-ruiz/self_fourier_shell_correlation"
from denoising.volume.gaussian import Monochrome_Denoising as GD # pip install --upgrade --force-reinstall "denoising @ git+https://github.com/vicente-gonzalez-ruiz/denoising"
from denoising.utils import get_gaussian_kernel # pip install --upgrade --force-reinstall "denoising @ git+https://github.com/vicente-gonzalez-ruiz/denoising"
import denoising.utils as utils # pip install --upgrade --force-reinstall "denoising @ git+https://github.com/vicente-gonzalez-ruiz/denoising"
import numpy as np
import logging

In [ ]:
vol_name = "empiar10311_stack_crop"
vol_extension = ".mrc"
vols_path = "/shared/vols/"

In [ ]:
from collections import namedtuple # Comes with the PSL
Args = namedtuple("args", ["vol"])
args = Args(vols_path + vol_name + vol_extension)

In [ ]:
with mrcfile.open(args.vol, permissive=True) as mrc:
    vol= mrc.data

In [ ]:
vol.shape

In [ ]:
vol = vol[0:100, 0:100, 0:100]

In [ ]:
#Z_dim = vol.shape[0]
#Z2 = Z_dim//2
#plt.imshow(vol[Z2], cmap="gray")

In [ ]:
def get_kernel_coeffs(sigma):
    sigmas = np.array([sigma, sigma, sigma])
    kernel = [None]*3
    kernel[0] = get_gaussian_kernel(sigmas[0])
    kernel[1] = get_gaussian_kernel(sigmas[1])
    kernel[2] = get_gaussian_kernel(sigmas[2])
    return kernel

In [ ]:
logging.basicConfig(format="[%(filename)s:%(lineno)s %(funcName)s()] %(message)s")
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
denoiser = GD(logger)

In [ ]:
curve = []
sigmas = [0.1, 0.5, 0.6, 0.75, 1.0, 1.5, 2.0, 3.0]
for i in sigmas:
    total_c_avg = 0
    kernel = get_kernel_coeffs(i)
    denoised_vol = denoiser.filter(vol, kernel)
    freq, c_avg = fsc.get_FSC_curve(vol, denoised_vol)
    #print(freq.shape, c_avg.shape)
    curve.append(c_avg)
    print(i, end=' ')

In [ ]:
plt.title("GD(" + vol_name + ", $\sigma$)")
plt.xlabel("Spatial Frequency")
plt.ylabel("Fourier Shell Correlation")
for i in range(len(sigmas)):
    plt.plot(freq, curve[i], label=f"$\sigma$={sigmas[i]}")
plt.legend()
plt.savefig(vol_name + "_GD__VS__FSC" + ".svg")
plt.show()

In [ ]:
curve = []
sigmas = [0.1, 0.5, 0.6, 0.75, 1.0, 1.5, 2.0]
for i in sigmas:
    total_c_avg = 0
    kernel = get_kernel_coeffs(i)
    denoised_vol = denoiser.filter(vol, kernel)
    freq, c_avg = fsc.get_SFSC_curve(denoised_vol)
    curve.append(c_avg)
    print(i, end=' ')

In [ ]:
plt.title("GD(" + vol_name + ", $\sigma$)")
plt.xlabel("Spatial Frequency")
plt.ylabel("Self Fourier Shell Correlation")
for i in range(len(sigmas)):
    plt.plot(freq, curve[i], label=f"$\sigma$={sigmas[i]}")
plt.legend()
plt.savefig(vol_name + "_GD__VS__SFSC" + ".svg")
plt.show()

## Why the SFSC curves go up with the frequency?

In [ ]:
np.max(vol)

In [ ]:
curve = []
sigmas = [0.1, 0.5, 0.6, 0.75, 1.0, 1.5, 2.0, 3.0]
for i in sigmas:
    total_c_avg = 0
    noise_1 = utils.gaussian_noise(vol.shape, sigma=100)
    noise_2 = utils.gaussian_noise(vol.shape, sigma=100)
    noisy_vol_1 = vol + noise_1
    noisy_vol_2 = vol + noise_2
    kernel = get_kernel_coeffs(i)
    denoised_vol_1 = denoiser.filter(noisy_vol_1, kernel)
    denoised_vol_2 = denoiser.filter(noisy_vol_2, kernel)
    freq, c_avg = fsc.get_FSC_curve(denoised_vol_1, denoised_vol_2)
    #print(freq.shape, c_avg.shape)
    curve.append(c_avg)
    print(i, end=' ')

In [ ]:
plt.title("GD(" + vol_name + ", $\sigma$)")
plt.xlabel("Spatial Frequency")
plt.ylabel("Fourier Shell Correlation")
for i in range(len(sigmas)):
    plt.plot(freq, curve[i], label=f"$\sigma$={sigmas[i]}")
plt.legend()
plt.savefig(vol_name + "_GD__VS__SFSC" + ".svg")
plt.show()

FSC has the same behaviour